# Módulo 5 Sprint

## Tema

**Aprendizaje de máquina supervisado**


*Objetivo del proyecto (Competencias del módulo)*


**Elaborar un modelo predictivo a partir de un set de datos utilizando técnicas de aprendizaje de máquina supervisado implementados en lenguaje Python para resolver un problema.**


# Descripción del ejercicio


## Contexto


Utilizaremos un set de datos de las policias de New York del año 2009 y 2010. (2009_1perc.csv y 2010_1perc.csv) los cuales nos darán información de los procedimientos policiales realizados. Además se le entregará el diccionario de variables para que pueda consultar que significa cada categoría dentro de las variables.


La variable respuesta 'arstmade' informa si los procedimientos policiales han terminado en arresto o no, y el objetivo será realizar un modelo de Machine Learning para predecir si un futuro procedimiento terminará en arresto.

**1.- Enliste todas las librerías que utilizará**

*Nota: Se recomienda ir actualizando la lista conforme las necesidades vaya teniendo durante el desarrollo de la prueba*


**2.- Importación y revisión de los datos**  *Importe ambos sets. Dado que la fuente de datos proviene de la misma base, tienen las mismas columnas. Consolide ambos sets y reporte una exploración básica de los datos (número de filas/columnas, tipos de datos, estadísticas básicas, casos perdidos)*

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



In [36]:
import warnings
warnings.filterwarnings(action='ignore') 

variable_respuesta = 'arstmade'

In [3]:
pro_pol_2009 = pd.read_csv('2009_1perc.csv')
pro_pol_2010 = pd.read_csv('2010_1perc.csv')

df_procedimientos_policiales = pd.concat([pro_pol_2009, pro_pol_2010]).reset_index()
df_procedimientos_policiales = df_procedimientos_policiales.drop(columns=['index'])


df_procedimientos_policiales.head()


,Unnamed: 0,year,pct,ser_num,datestop,timestop,recstat,inout,trhsloc,perobs,...,zip,addrpct,sector,beat,post,xcoord,ycoord,dettypcm,linecm,detailcm
0,178048,2009,41,1779,4032009,130,A,O,P,1.0,...,,41,G,7,,1013067,0238633,CM,1,20
1,498873,2009,108,5805,10292009,1050,A,O,P,3.0,...,,108,J,,,1012043,0212157,CM,1,14
2,463573,2009,43,8340,10062009,1450,1,O,P,1.0,...,,43,E,,,1017599,0240200,CM,1,20
3,43626,2009,77,932,1232009,1843,A,O,P,5.0,...,,77,J,4,35,1002625,0183442,CM,1,24
4,563921,2009,110,11224,12132009,1655,A,O,P,3.0,...,,110,H,,,1024535,0209890,CM,1,20


In [4]:
df_procedimientos_policiales = df_procedimientos_policiales.replace(' ', np.nan)


In [5]:

dimensiones_df = df_procedimientos_policiales.shape   # (11825, 112)
info_df = df_procedimientos_policiales.info()  # dtypes: float64(7), int64(16), object(89)

categorical = [var for var in df_procedimientos_policiales.columns if df_procedimientos_policiales[var].dtype=='O']
integer_variable = [var for var in df_procedimientos_policiales.columns if df_procedimientos_policiales[var].dtype=='int64']
float_variable = [var for var in df_procedimientos_policiales.columns if df_procedimientos_policiales[var].dtype=='float64']

df_categorical = df_procedimientos_policiales[categorical]
df_integer_variable = df_procedimientos_policiales[integer_variable]
df_float_variable = df_procedimientos_policiales[float_variable]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11825 entries, 0 to 11824
Columns: 112 entries, Unnamed: 0 to detailcm
dtypes: float64(7), int64(16), object(89)
memory usage: 10.1+ MB


In [7]:
df_categorical.head()

,recstat,inout,trhsloc,crimsusp,typeofid,explnstp,othpers,arstmade,arstoffn,sumissue,...,city,addrpct,sector,beat,post,xcoord,ycoord,dettypcm,linecm,detailcm
0,A,O,P,CPW,P,Y,N,N,NaN,N,...,BRONX,41,G,7,NaN,1013067,0238633,CM,1,20
1,A,O,P,BURG,P,Y,Y,N,NaN,N,...,QUEENS,108,J,NaN,NaN,1012043,0212157,CM,1,14
2,1,O,P,MISD,V,Y,N,N,NaN,N,...,BRONX,43,E,NaN,NaN,1017599,0240200,CM,1,20
3,A,O,P,MIS,P,Y,N,N,NaN,N,...,BROOKLYN,77,J,4,35,1002625,0183442,CM,1,24
4,A,O,P,CPW,P,Y,Y,N,NaN,N,...,QUEENS,110,H,NaN,NaN,1024535,0209890,CM,1,20


In [8]:
df_integer_variable.head()

,Unnamed: 0,year,pct,ser_num,datestop,timestop,perstop,compyear,comppct,repcmd,revcmd,dob,age,ht_feet,ht_inch,weight
0,178048,2009,41,1779,4032009,130,2,0,0,41,41,7111984,24,5,9,145
1,498873,2009,108,5805,10292009,1050,5,0,0,108,108,12151993,15,5,5,140
2,463573,2009,43,8340,10062009,1450,1,0,0,43,43,7031987,21,6,3,220
3,43626,2009,77,932,1232009,1843,5,0,0,77,77,3131986,22,5,5,120
4,563921,2009,110,11224,12132009,1655,10,0,0,110,110,9221991,18,5,6,165


In [9]:
df_float_variable.head()

,perobs,othfeatr,rescode,premtype,aptnum,state,zip
0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,3.0,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,5.0,NaN,NaN,NaN,NaN,NaN,NaN
4,3.0,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
df_int_nulos = df_integer_variable.isnull().sum()
df_flo_nulos = df_float_variable.isnull().sum()
df_cat_nulos = df_categorical.isnull().sum()


df_nulos = pd.concat([df_int_nulos, df_cat_nulos, df_flo_nulos])
df_nulos = df_nulos[(df_nulos != 0)]

columna = ['Cantidad']

df_nulos = pd.DataFrame(df_nulos, columns=columna)

df_nulos['Porcentaje'] = round((df_nulos / df_procedimientos_policiales.shape[0]) * 100, 2)

correlation = df_procedimientos_policiales[variable_respuesta].corr(df_procedimientos_policiales['columna_1'])


df_nulos.sort_values('Cantidad', ascending=False)

,Cantidad,Porcentaje
zip,11825,100.00
state,11825,100.00
aptnum,11825,100.00
premtype,11825,100.00
rescode,11825,100.00
othfeatr,11825,100.00
officrid,11664,98.64
arstoffn,11078,93.68
sumoffen,11023,93.22
post,10651,90.07


Unnamed: 0    0
year          0
pct           0
ser_num       0
datestop      0
timestop      0
perstop       0
compyear      0
comppct       0
repcmd        0
revcmd        0
dob           0
age           0
ht_feet       0
ht_inch       0
weight        0
dtype: int64

In [11]:
df_float_variable.isnull().sum()

perobs          0
othfeatr    11825
rescode     11825
premtype    11825
aptnum      11825
state       11825
zip         11825
dtype: int64

In [13]:
pd.set_option('display.max_rows', None)
df_categorical.isnull().sum()

recstat         0
inout           0
trhsloc       205
crimsusp        0
typeofid        0
explnstp        0
othpers         0
arstmade        0
arstoffn    11078
sumissue        0
sumoffen    11023
offunif         0
officrid    11664
frisked         0
searched        0
contrabn        0
adtlrept        0
pistol          0
riflshot        0
asltweap        0
knifcuti        0
machgun         0
othrweap        0
pf_hands        0
pf_wall         0
pf_grnd         0
pf_drwep        0
pf_ptwep        0
pf_baton        0
pf_hcuff        0
pf_pepsp        0
pf_other        0
radio           0
ac_rept         0
ac_inves        0
rf_vcrim        0
rf_othsw        0
ac_proxm        0
rf_attir        0
cs_objcs        0
cs_descr        0
cs_casng        0
cs_lkout        0
rf_vcact        0
cs_cloth        0
cs_drgtr        0
ac_evasv        0
ac_assoc        0
cs_furtv        0
rf_rfcmp        0
ac_cgdir        0
rf_verbl        0
cs_vcrim        0
cs_bulge        0
cs_other        0
ac_incid  